# Portfolio Optimization with Scipy
This is an example of how to use **scipy.optimize.minimize** to perform portfolio optimization.  The problem we are trying to solve with Portfolio Optimization is how to determine the optimum asset mix to optimize certai portfolio objectives

*  Minimize Volitility
*  Maximize Returns
*  Maximize the Portfolio's Sharpe Ratio 


## A. Calculate Earnings for Assests in Portfolio

Create list of tickers symbols of assets in the portfolio to optimize.  Also, define the **start** date and **end** date to pull.

In [1]:
import datetime

tickers = sorted(['VWILX','VPMAX','VFORX','VWNEX','VIIIX','VEMPX','VGHAX','VWELX','VIPSX'])

start = datetime.datetime(2016, 8, 1)
end = datetime.datetime.today()

### Download Daily Data
We will use **pandas_datareader** to download the daily **Adjusted Close** price of the assets in the porfolio from Google.

In [2]:
import pandas as pd
import pandas_datareader.data as web
import fix_yahoo_finance as yf

yf.pdr_override()

TICKER_DATA = yf.download(tickers, start=start)
TICKER_DATA.Close.head()

C:\Users\rick\Anaconda3\lib\site-packages\fix_yahoo_finance\__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and will be removed in future versions.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


[*********************100%***********************]  9 of 9 downloaded

,VEMPX,VFORX,VGHAX,VIIIX,VIPSX,VPMAX,VWELX,VWILX,VWNEX
Date,,,,,,,,,
2016-08-01,168.089996,30.230000,92.050003,198.600006,13.67,108.510002,38.919998,69.580002,65.930000
2016-08-02,166.009995,30.059999,91.849998,197.350006,13.61,108.160004,38.759998,69.169998,65.349998
2016-08-03,167.279999,30.070000,91.980003,198.020004,13.64,108.199997,38.860001,68.849998,65.980003
2016-08-04,167.399994,30.150000,91.870003,198.110001,13.70,108.459999,38.889999,69.120003,66.029999
2016-08-05,169.089996,30.350000,91.589996,199.820007,13.66,109.620003,39.070000,69.790001,66.849998


### Define the Risk Free Rate
We will use the expected daily returns of the **S&P 500**

In [3]:
RISK_FREE_DATA = web.DataReader('^GSPC', 'yahoo', min(TICKER_DATA.Close.index)).Close
RISK_FREE_RATE = RISK_FREE_DATA.pct_change()[1:].mean() 
RISK_FREE_RATE

0.0004951380049318706

### Calculate Business Days for the past Year¶
As a proxy for tradig days

In [4]:
from datetime import timedelta
BDAYS = RISK_FREE_DATA.loc[end - timedelta(days=365):end].asfreq('B').count()
BDAYS

252

### Plot Prices and Returns

In [5]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category20 as plt
output_notebook()

p = figure( title = 'Portfolio Asset Prices' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.toolbar.active_drag = None

for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(TICKER_DATA.Close).index, y=pd.DataFrame(TICKER_DATA.Close)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

In [6]:
p = figure( title = 'Portfolio Normalized Returns' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.toolbar.active_drag = None

p.line(RISK_FREE_DATA.index, RISK_FREE_DATA / RISK_FREE_DATA.iloc[0] - 1 , 
       color='red', line_width=5, legend='S&P 500', alpha=0.25)

NORMALIZED_RETURNS = TICKER_DATA.Close / TICKER_DATA.Close.iloc[0] - 1
for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(NORMALIZED_RETURNS).index, y=pd.DataFrame(NORMALIZED_RETURNS)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

### Calculate Daily Assets Returns
Determine the percent change from one day to the next, as daily return, and save it to the **RETURNS** data frame.

$$ \hbox{Expected Returns} = \frac{Close - Close_{-1} }{Close_{-1} } $$

In [7]:
RETURNS = TICKER_DATA['Close'].pct_change()[1:]
RETURNS.tail()

,VEMPX,VFORX,VGHAX,VIIIX,VIPSX,VPMAX,VWELX,VWILX,VWNEX
Date,,,,,,,,,
2017-09-18,0.005570,0.001738,-0.003117,0.001486,-0.000752,0.001862,0.001192,0.007903,0.003682
2017-09-19,-0.003405,0.001445,-0.007704,0.001134,-0.001506,0.001703,0.001667,0.002793,0.000000
2017-09-20,0.003570,0.000000,0.006864,0.000610,-0.003017,0.000232,0.000951,-0.002999,0.002883
2017-09-21,-0.001982,-0.002309,-0.004806,-0.008276,-0.008321,-0.000618,-0.006415,-0.002149,0.000261
2017-09-22,0.003309,0.001157,0.003481,0.000659,0.003051,0.004871,0.001913,0.001077,0.001829


In [8]:
p = figure( title = 'Asset Returns' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Returns'
p.toolbar.active_drag = None

for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(RETURNS).index, y=pd.DataFrame(RETURNS)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

### Calculate the average daily returns
For each asset in the portfolio, we need to determine the average returns by calculating the mean of the observed days.

In [9]:
EXPECTED_RETURNS = RETURNS.mean()
EXPECTED_RETURNS

VEMPX    0.000576
VFORX    0.000478
VGHAX   -0.000072
VIIIX    0.000489
VIPSX   -0.000129
VPMAX    0.000647
VWELX    0.000262
VWILX    0.001025
VWNEX    0.000545
dtype: float64

These returns can be annualized by $$ (( Daily Returns + 1)^{Business Days} - 1) * 100% $$

In [10]:
((EXPECTED_RETURNS + 1)**BDAYS -1)*100

VEMPX    15.626115
VFORX    12.809494
VGHAX    -1.788308
VIIIX    13.097467
VIPSX    -3.205198
VPMAX    17.706888
VWELX     6.823528
VWILX    29.463929
VWNEX    14.727450
dtype: float64

### Calculate the covariance matrix of the returns
The variance/covariance matrix provides a measure of volatility for the assets (and between assets) in the portfolio.

In [11]:
COVARIANCE = RETURNS.cov()
COVARIANCE

,VEMPX,VFORX,VGHAX,VIIIX,VIPSX,VPMAX,VWELX,VWILX,VWNEX
VEMPX,0.000052,0.000028,0.000029,3.180007e-05,-1.282922e-06,0.000036,0.000019,0.000030,0.000039
VFORX,0.000028,0.000022,0.000020,2.201666e-05,1.209696e-06,0.000024,0.000014,0.000025,0.000025
VGHAX,0.000029,0.000020,0.000062,2.480551e-05,6.337139e-06,0.000039,0.000019,0.000028,0.000032
VIIIX,0.000032,0.000022,0.000025,2.638240e-05,8.636098e-07,0.000030,0.000017,0.000026,0.000030
VIPSX,-0.000001,0.000001,0.000006,8.636098e-07,9.799777e-06,0.000004,0.000003,0.000005,0.000002
VPMAX,0.000036,0.000024,0.000039,3.004491e-05,4.230192e-06,0.000045,0.000021,0.000032,0.000038
VWELX,0.000019,0.000014,0.000019,1.700169e-05,3.436312e-06,0.000021,0.000013,0.000017,0.000022
VWILX,0.000030,0.000025,0.000028,2.563784e-05,4.731225e-06,0.000032,0.000017,0.000043,0.000029
VWNEX,0.000039,0.000025,0.000032,3.008046e-05,1.935841e-06,0.000038,0.000022,0.000029,0.000043


## B. Create a coupe of utility functions


Define a function (**RANDOM_PORTFOLIO_WEIGHTS**) to generate a random mixed portfolio and a function (**PORTFOLIO**) to calculate the portfolio's **mean returns** ($\mu = w * p^T$) and **standard deviation** ($\sigma = \sqrt{w^T * C * w}$).  Where $w$ is a **vector of asset weights** and $C$ is a **covariance matrix**.


In [12]:
import numpy as np


def RANDOM_PORTFOLIO_WEIGHTS(ticker_count):
    ''' 
    Create a portfolio mix without shorts
    '''
    w = np.random.uniform(0,1,ticker_count)
    return w / sum(w)



def PORTFOLIO( weights ):
    ''' 
    Calculates and returns the portfolio 
        Avgerage Daily Returns (mu),  
        Standard deviation of the Daily Returns (sigma), and
        Sharpe Ratio
    '''
    p = np.asmatrix(EXPECTED_RETURNS)
    w = np.asmatrix(weights)
    C = np.asmatrix(COVARIANCE)
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    sharpe = (mu - RISK_FREE_RATE)/sigma
    annualized_returns = ((mu[0,0] + 1)**BDAYS - 1) * 100
    
    return [mu[0,0], sigma[0,0], sharpe[0,0], annualized_returns]
    

Generate a set of **n_portfolios** portfolios, with random mixed parts of the assets in **tickers**, and use the **PORTFOLIO** function to obtain *Expected Daily Returns*, *Standard Deviation* of the Daily Returns, and the portfolio's *Sharpe ratio*.

In [13]:
n_portfolios = 10000
means, stds, sharpe, annu_returns = np.column_stack([
    PORTFOLIO( RANDOM_PORTFOLIO_WEIGHTS(len(tickers)) ) 
    for _ in range(n_portfolios)
])

### Calculate the Efficient Frontier
The Efficient Frontier is the line of maximum returns as a function of volatility for a mix of assets.

http://www.quantandfinancial.com/2013/07/mean-variance-portfolio-optimization.html

In [14]:
from scipy.optimize import minimize


def miniVolatilityFitness(W, r):
    '''For given level of return r, find weights which minimizes
    portfolio variance.'''
    mean, var, _, _ = PORTFOLIO(W)
    # Big penalty for not meeting stated portfolio return 
    # effectively serves as optimization constraint
    penalty = 1000 * abs(mean-r)       
    return var + penalty
    
def solve_frontier(R, C):
    '''
    Given risk-free rate, assets returns and covariances, this function calculates 
    min-var frontier and returns its [x,y] points in two arrays.
    '''
    frontier_mean, frontier_var, frontier_sharpe, frontier_aret = [], [], [], []
    n = len(R)  # Number of assets in the portfolio
    W = np.ones([n])/n     # start optimization with equal weights
    b_ = [(0,1) for i in range(n)]
    c_ = ({'type':'eq', 'fun': lambda W: sum(W)-1. })
    
    # Iterate through the range of returns on Y axis
    for r in np.linspace(min(R), max(R), num=50): 
        optimized = minimize(fun=miniVolatilityFitness, x0=W, args=r, method='SLSQP', 
                             constraints=c_, bounds=b_, options={'ftol': 1e-10})   
        if not optimized.success: 
            raise BaseException(optimized.message)
            
        # add point to the min-var frontier [x,y] = [optimized.x, r]
        mu, sig, shp, aret = PORTFOLIO(optimized.x)
        frontier_mean.append(mu)
        frontier_var.append(sig)
        frontier_sharpe.append(shp)
        frontier_aret.append(aret)
        
    return frontier_mean, frontier_var, frontier_sharpe, frontier_aret

C:\Users\rick\Anaconda3\lib\importlib\_bootstrap.py:205: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\rick\Anaconda3\lib\importlib\_bootstrap.py:205: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [15]:
frontier_mean, frontier_var, frontier_sharpe, frontier_aret = solve_frontier(EXPECTED_RETURNS, COVARIANCE)

### Plot the Markowitz Bullet of Random Mixed Weights Portfolios

Use the expected returns and volatility of the random portfolios to draw the Markowitz Bullet of the porfolios.

In [16]:
p = figure( title = 'Markowitz Bullet for set of Random Portfolios' , plot_width=980 )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
p.toolbar.active_drag = None
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')

show(p)

## D. Find the Minimum Volatility Portfolio

The **Scipy minimize** function will need a function with one objective (Variance).  The **getVolatility** function wraps the **PORTFOLIO** function and only returns the expected variance.

In [17]:
def getVolatility(mix):
    return( PORTFOLIO(mix)[1] )

In [18]:
from scipy.optimize import minimize

x0 = np.ones(len(tickers)) 


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1

OPT_VOLATILITY = minimize(getVolatility, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-8})


print( "Lowest Daily Volatility {}".format( np.round(OPT_VOLATILITY.fun, 4) ) )
OPT_VOLATILITY_MIX = pd.DataFrame(np.round( OPT_VOLATILITY.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_VOLATILITY_MIX[OPT_VOLATILITY_MIX['Percent of Portfolio']>0]

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0026724842199031067
            Iterations: 22
            Function evaluations: 242
            Gradient evaluations: 22
Lowest Daily Volatility 0.0027


,Percent of Portfolio
VFORX,13.52
VIPSX,62.57
VWELX,23.90


In [19]:
min_volatility_mean , min_volatility_std , min_volatility_sharpe , min_volatility_annualized = PORTFOLIO( OPT_VOLATILITY.x ) 
print( 'Markowitz Bullet for Portfolio with the minimum volatility of {1}, daily returns of {0}, and Sharpe ratio of {2}'.\
           format(np.round(min_volatility_mean,4) , np.round(min_volatility_std,4) , np.round(min_volatility_sharpe,4) ) )

Markowitz Bullet for Portfolio with the minimum volatility of 0.0027, daily returns of 0.0, and Sharpe ratio of -0.1679


In [20]:
p = figure( title = 'Portfolio with the minimum volatility of {1}, annualized returns of {0}%, and Sharpe ratio of {2}'.\
           format(np.round(min_volatility_annualized,2) , np.round(min_volatility_std,4) , np.round(min_volatility_sharpe,4) ) , 
           plot_width=980  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns' 
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.circle(x=min_volatility_std, y=min_volatility_mean, fill_alpha=0.77 , size=14 , color='red' )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')
p.toolbar.active_drag = None
show(p)

## E. Efficient Portfolio for the highest daily returns

In [21]:
def getReturns(mix):
    return( -PORTFOLIO(mix)[0] )

In [22]:
from scipy.optimize import minimize

x0 = np.ones(len(tickers))  


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES BETWEEN -1 AND 1

OPT_RETURNS = minimize(getReturns, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-7})


print( "Highest Daily Returns {}".format( np.round(-OPT_RETURNS.fun, 4) ) )
OPT_RETURNS_MIX = pd.DataFrame(np.round( OPT_RETURNS.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_RETURNS_MIX[OPT_RETURNS_MIX['Percent of Portfolio']>0]

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.0010252558433794517
            Iterations: 33
            Function evaluations: 363
            Gradient evaluations: 33
Highest Daily Returns 0.001


,Percent of Portfolio
VWILX,100.0


In [23]:
target_return_mean , target_return_std , target_return_sharpe , target_return_annualized = PORTFOLIO( OPT_RETURNS.x ) 
print( 'Markowitz Bullet for Portfolio with maximum daily returns of {0}, sigma of {1}, and Sharpe ratio {2}'.\
           format(np.round(target_return_mean,4) , np.round(target_return_std,4) , np.round(target_return_sharpe,5) ) )

Markowitz Bullet for Portfolio with maximum daily returns of 0.001, sigma of 0.0066, and Sharpe ratio 0.0805


In [24]:
p = figure( title = 'Portfolio with maximum annual returns of {0}%, sigma of {1}, and Sharpe ratio {2}'.\
           format(np.round(target_return_annualized,2) , np.round(target_return_std,4) , np.round(target_return_sharpe,4) ) , 
           plot_width=980  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.circle(x=target_return_std, y=target_return_mean, fill_alpha=0.77 , size=14 , color='red' )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')
p.toolbar.active_drag = None
show(p)

## F. Optimum Sharpe Ratio

http://www.investopedia.com/terms/s/sharperatio.asp

The **Scipy minimize** function will need a function with one objective (-Sharpe ratio).  The **getSharpe** function wraps the **PORTFOLIO** function and only returns the Sharpe ratio.

In [25]:
def getSharpe(mix):
    return( -PORTFOLIO(mix)[2] )

In [26]:
from scipy.optimize import minimize


x0 = np.ones(len(tickers)) 


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1

OPT_SHARPE = minimize(getSharpe, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-7})


print( "Highest Sharpe Ratio {}".format( np.round(-OPT_SHARPE.fun, 4) ) )
OPT_SHARPE_MIX = pd.DataFrame(np.round( OPT_SHARPE.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_SHARPE_MIX[OPT_SHARPE_MIX['Percent of Portfolio']>0]

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.08050475716456357
            Iterations: 8
            Function evaluations: 88
            Gradient evaluations: 8
Highest Sharpe Ratio 0.0805


,Percent of Portfolio
VWILX,100.0


In [27]:
best_sharpe_return , best_sharpe_std , best_sharpe , best_sharpe_annualized_return = PORTFOLIO( OPT_SHARPE.x.T ) 
print('Markowitz Bullet for Portfolio with maximum Sharpe ratio of {2}, return of {0}, and sigma of {1}'.\
           format(np.round(best_sharpe_return,4) , np.round(best_sharpe_std,4) , np.round(best_sharpe,4)) )

Markowitz Bullet for Portfolio with maximum Sharpe ratio of 0.0805, return of 0.001, and sigma of 0.0066


In [28]:
p = figure( title = 'Portfolio with maximum Sharpe ratio of {2}, annualized returns of {0}%, and sigma of {1}'.\
           format(np.round(best_sharpe_annualized_return,2) , np.round(best_sharpe_std,4) , np.round(best_sharpe,4) ) , 
           plot_width=980  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.circle(x=best_sharpe_std, y=best_sharpe_return, fill_alpha=0.77 , size=14 , color='red' )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')
p.toolbar.active_drag = None
show(p)